In [1]:
import pandas as pd
import numpy as np

In [1]:
class ReviewModel():
    #y_after_predict: 3-dimension array
    #y_after_predict[model type index][model series squecence in specific hypa index][y_value_after_predict index]
    y_after_predict = [[[]]]
    val_y = pd.Series([],dtype=pd.StringDtype()) 
    # DistanceType: 'Hausdoff'
    # DistanceType: 'r2', 'mse'
    DistanceType = ''
    baseline_preds = 0.0
    bestModel = [[]] # binary, mark index of the best
    #Distance between y_after_predict and val_y
    distance = [[]]
    #Index of <Model type> element in y_after_predict have the best result
    best = 0
    def __init__(self,val_y,y_after_predict,baseline_preds = None, DistanceType = 'Hausdoff'):
        self.DistanceType = DistanceType
        self.val_y = val_y
        self.y_after_predict = y_after_predict
        self.baseline_preds = baseline_preds
#         self.bestModel.reshape(y_after_predict.shape)
#         self.minDistance.reshape(y_after_predict.shape)
        pass

    
###
# Hausdoff Distance
###
    def HausdoffDistanceBetween_xAndA(self,x,A):
        minn = float('inf')
        for a in A:
            if abs(x-a) < minn:
                minn = abs(x-a)
        return minn
    def HausdoffDistanceBetween_AAndB(self,A,B):
        maxx = -float('inf')
        max1 = -float('inf')
        max2 = -float('inf')
        for a in A:
            temp = self.HausdoffDistanceBetween_xAndA(a,B)
            if temp > max1:
                max1 = temp
        for b in B:
            temp = self.HausdoffDistanceBetween_xAndA(b,A)
            if temp > max2:
                max2 = temp
        if max1 > max2:
            maxx = max1
        else:
            maxx = max2
        return maxx
        pass
###
# R^2
###
    def compute_mse(self,y, preds):
        return ((y - preds) ** 2).mean()
    def compute_rr(self,y, preds):
        return 1 - self.compute_mse(y, preds) / self.compute_mse(y, self.baseline_preds)
    
###
#Calculate
###
    def chooseTheBestHyperParameter(self):
        distance = [0]
        binaryArray=[0]
        self.distance.clear()
        self.bestModel.clear()
        for modelType in self.y_after_predict:
            distance.clear()
            binaryArray.clear()
            for modelHypa in modelType:
                if self.DistanceType == 'Hausdoff':
                    distance.append(self.HausdoffDistanceBetween_AAndB(modelHypa,self.val_y))
                elif self.DistanceType == 'r2':
                    distance.append(self.compute_rr(self.val_y,modelHypa))
                elif self.DistanceType == 'mse':
                    distance.append(self.compute_mse(self.val_y,modelHypa))
                else:
                    distance.append(self.HausdoffDistanceBetween_AAndB(modelHypa,self.val_y))
                binaryArray.append(0)
            print(distance)
            self.distance.append(distance.copy())
            if self.DistanceType == 'Hausdoff' or self.DistanceType == 'mse':
                binaryArray[distance.index(min(distance))] = 1
            elif self.DistanceType == 'r2':
                binaryArray[distance.index(max(distance))] = 1
            else:
                binaryArray[distance.index(min(distance))] = 1
            self.bestModel.append(binaryArray.copy())
    
    def chooseTheBestModel(self):
        i = 0
        distance = []
        distance.clear()
        for modelType in self.y_after_predict:
            # print series have the best result in this model type
#             print(modelType[self.bestModel[i].index(max(self.bestModel[i]))])
#             modelType[self.bestModel[i].index(max(self.bestModel[i]))]
            # print min distance in this model type
#             print(self.distance[i][self.bestModel[i].index(max(self.bestModel[i]))])
            distance.append(self.distance[i][self.bestModel[i].index(max(self.bestModel[i]))])
            i += 1
        if self.DistanceType == 'Hausdoff' or self.DistanceType == 'mse':
                self.best = distance.index(min(distance))
        elif self.DistanceType == 'r2':
                self.best = distance.index(max(distance))
        else:
                self.best = distance.index(min(distance))
        
        pass
    def calculate(self):
        self.chooseTheBestHyperParameter()
        self.chooseTheBestModel()
        return self.bestModel, self.best
            
    def __del__(self):
        pass
    pass

NameError: name 'pd' is not defined

In [122]:
baseline_preds = 125.14696927760863
val_y = np.random.randint((1000),size=(389))
y_after_predict = np.random.randint((2*398*5),size=(2,5,389))

In [112]:
test = ReviewModel(val_y, y_after_predict)
test1 = ReviewModel(val_y, y_after_predict,baseline_preds,'r2')
# test.val_y = pd.Series([1,2,3])

In [113]:
test.chooseTheBestHyperParameter()

[2967, 2968, 2974, 2976, 2979]
[2964, 2950, 2972, 2980, 2981]


In [114]:
test.distance

[[2967, 2968, 2974, 2976, 2979], [2964, 2950, 2972, 2980, 2981]]

In [115]:
test.bestModel

[[1, 0, 0, 0, 0], [0, 1, 0, 0, 0]]

In [116]:
test.chooseTheBestModel()

In [117]:
test.best

1

In [118]:
binaryArray,bestIndex = test1.calculate()

[-14.707678546763718, -15.731831208965179, -15.617842593965989, -16.227971817782525, -15.113278856403607]
[-14.846385537348471, -15.696350217437104, -14.188260137814883, -15.923347349212975, -16.74479674363721]


In [119]:
binaryArray

[[1, 0, 0, 0, 0], [0, 0, 1, 0, 0]]

In [120]:
bestIndex

1

In [121]:
test1.distance

[[-14.707678546763718,
  -15.731831208965179,
  -15.617842593965989,
  -16.227971817782525,
  -15.113278856403607],
 [-14.846385537348471,
  -15.696350217437104,
  -14.188260137814883,
  -15.923347349212975,
  -16.74479674363721]]